In [2]:
#Imports
import pandas as pd
import numpy as np
from patsy import dmatrices
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor



In [3]:
df = pd.read_csv('vif.csv')
df.dropna()
df1 = df._get_numeric_data() #drop non-numeric cols

df1.head()

,Unnamed: 0,y,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,X11,X12,X13,X14,X15
0,1,-38.227132,-0.058972,-0.206567,0.800651,0.775224,1.967855,0.378186,3.341408,-2.317802,-2.515558,-3.988083,-3.862908,4.879319,1.250428,3.247059,-0.035007
1,2,-37.675237,-1.627567,-1.613886,0.105250,1.380120,5.814475,2.651844,3.965394,-0.366479,-4.974998,0.184738,-5.380564,-0.577288,-4.827699,6.719420,1.811248
2,3,-57.970633,-3.808515,3.791119,3.994495,-0.104410,0.663476,1.213484,2.488109,-0.205445,-1.870075,-1.035900,-1.350974,-1.029284,2.840111,-0.848216,0.403430
3,4,-12.646060,-5.383439,2.249870,3.844734,-2.479057,1.436955,1.951578,1.138720,-0.522607,-2.962268,-0.273728,-0.222272,-2.935165,-2.345417,-1.263170,3.167941
4,5,-16.220116,-5.619411,1.904953,5.689017,-2.080941,-5.967984,0.815694,0.054039,0.323300,-1.827142,0.984362,-1.380817,-4.432278,5.542334,-4.785662,-0.584143


In [4]:
df2=df1.drop('y',axis=1)

In [5]:
from statsmodels.stats.outliers_influence import variance_inflation_factor  

def calculate_vif_(X):
    variables = list(X.columns)
    vif = {variable:variance_inflation_factor(exog=X.values, exog_idx=ix) for ix,variable in enumerate(list(X.columns))}
    return vif

In [6]:
vifDict = calculate_vif_(df2)

In [7]:
vifDict

{'Unnamed: 0': 1.056905722637264,
 'X1': 27.953159132110354,
 'X10': 51.486387838720887,
 'X11': 34.078312249888448,
 'X12': 44.575303135869781,
 'X13': 12.168985404919226,
 'X14': 74.97190313242082,
 'X15': 30.604729744222702,
 'X2': 36.850507536635142,
 'X3': 12.809747032810531,
 'X4': 50.894501539829449,
 'X5': 8.4351615177161818,
 'X6': 115.22890360004568,
 'X7': 68.235614935086758,
 'X8': 153.82341808366408,
 'X9': 48.407920684253426}

In [20]:
from statsmodels.stats.outliers_influence import variance_inflation_factor  

def calculate_vif_(X):
    variables = list(X.columns)
    vif = {variable:variance_inflation_factor(exog=X.values, exog_idx=ix) for ix,variable in enumerate(list(X.columns))}
    return vif

def loop_vif(data_frame,dep_var,vif_cut_off):
    data_frame.dropna(inplace = True)
    data_frame=data_frame.drop(dep_var,axis=1)
    data_frame = data_frame._get_numeric_data() #drop non-numeric cols
    
    
    max_vif = vif_cut_off +2
    
    while(max_vif >= vif_cut_off):
    
        vifDict = calculate_vif_(data_frame)
    
        vifDf = pd.DataFrame()
        vifDf['variables'] = vifDict.keys()
        vifDf['vifScore'] = vifDict.values()
        vifDf.sort_values(by=['vifScore'],ascending=False,inplace=True)
        vifDf.reset_index(inplace=True,drop=True)
        print(vifDf)
        max_vif = max(vifDf['vifScore'])
        
        if(max_vif > vif_cut_off):
            drop_var = vifDf[vifDf['vifScore'] == max(vifDf['vifScore'])]['variables'][0]
            print('Variable {} will be removed as VIF score {} is greater than {}'.format(drop_var,max_vif,vif_cut_off))
            data_frame.drop(drop_var,axis=1,inplace=True)
    return(data_frame.columns)


In [21]:
df = pd.read_csv('vif.csv')

In [22]:
df.columns

Index(['y', 'X1', 'X2', 'X3', 'X4', 'X5', 'X6', 'X7', 'X8', 'X9', 'X10', 'X11',
       'X12', 'X13', 'X14', 'X15'],
      dtype='object')

In [23]:
loop_vif(df,'y',5)

   variables    vifScore
0         X8  153.612375
1         X6  115.228883
2        X14   74.598818
3         X7   67.290179
4         X4   50.891559
5        X10   50.767700
6         X9   48.142476
7        X12   44.324220
8         X2   36.848908
9        X11   34.051869
10       X15   30.522286
11        X1   27.907582
12        X3   12.802736
13       X13   12.082014
14        X5    8.354233
Variable X8 will be removed as VIF score 153.6123750492101 is greater than 5
   variables   vifScore
0         X7  52.998266
1        X10  46.340609
2        X12  43.163684
3        X14  37.936287
4        X15  22.030004
5         X6  19.800169
6        X11  18.324388
7         X9  17.869381
8        X13  10.895628
9         X4   8.068404
10        X2   7.984754
11        X5   7.718239
12        X1   6.722488
13        X3   4.597275
Variable X7 will be removed as VIF score 52.99826643117125 is greater than 5
   variables   vifScore
0        X12  13.203847
1         X6   9.187639
2        X15  

Index(['X1', 'X2', 'X3', 'X4', 'X5', 'X9', 'X10', 'X11', 'X13', 'X14', 'X15'], dtype='object')